# <span style="color:cornflowerblue"> Gerald Jones, Kimon Swanson, Alex Hines</span>
# <span style="color:cornflowerblue"> IE 402 - Group project 3a - PPF :</span>
# <span style="color:cornflowerblue"> ISE522 Spg 22</span>

## Notebook Links:
1. [Data Display section](#Data-Display)
2. [Model Formulation](#Model-Formulation)
3. [Method Definitions](#Method-Definitions)
4. [Gurobi Implementation](#implementation)
5. [Solution Discussion](#solution)

## Problem Description:
> ### **Summary** 
Perfection Pet Foods (PPF), a co-manufacturer serving the pet food industry, seeks to optimize their management of empty bag inventory. They need us to develop an inventory control policy that will ***minimize cost***. 

> ### **About Perfection Pet Foods** 
Perfection Pet Foods’ extensive production and manufacturing capabilities are what set them apart from the competition. 
From super premium, non-grain pet food to mass consumer private label products, they consistently deliver the highest 
quality product to both retailers and consumers. Their plant in Visalia, CA is a state-of-the-art dry kibble production 
facility, bringing much needed co-pack capacity and capability to the west coast. The plan typically operates on a 24/7 
basis, with down days planned accordingly for preventative maintenance and to manage supply/demand balance. 

> ### **Keywords and concepts** 
> - Kibble: another word for the food itself.
> - A packaged bag of pet food is referred to as a finished good (FG) product.
> - Mono-kibble product: a FG product that consists of a single kibble.
> - Multi-kibble: a FG product that consists of a mixture of two or more kibbles.
> - Component formula: a single type of kibble
> - Master formula: the contents of a FG product that goes in a bag. A master formula may consist of a single component formula (mono-kibble) or multiple component formulas (multi-kibble)

> ### **Process Description** 
At PPF, pet food is manufactured in two stages: extrusion and pack out. 
> - ***Extrusion***. First, the pet food is extruded, taking in batched ingredients, through extruding machines which create the actual kibble. The plant has **three extruders**, and the **extrusion rate** is given in the data spreadsheet. (Following extrusion, product goes through ovens to dry to desired moisture, but this process may be ignored for this assignment.) It is **possible for an extruder to switch bins during a production run**. 
> - ***Pack out***. After the kibble is extruded, it is packaged into (sellable) bags, which **may be mono-kibble or multikibble**. There are **four pack lines**: **one for small bags (called the Parsons line)** and **three for medium and large bags** (called Umbra, Premier Tech, and Thiele, respectively). The **throughput rate for the packing process depending on the pack line as well as the kibble type**. The **pack lines can start pulling kibble before a bin is full.** There is no room on the pack lines for staging kibble.

> ### **Objective** 
Develop an inventory policy that **minimizes amount spent on bags while covering all forecasted production needs 
(including potential variability)** and not exceeding space constraints on site or at supplier.



## Other information 
* Yield loss
    - You should assume a 10% yield loss on bags when running. So, if the run is 200,000# of 40# Bags, bag 
      needs are 200,000/40 = 5,000 bags / (1 – 0.1) = 5,556 bags. 
* Variability
    - You should assume that actual bag needs could vary +/- 15% from forecasted need. Thus, you should make sure that the inventory level each product in each week does not drop below 0.15 times the demand for that week.
* Inventory Space Constraints
    - Total weeks-of-supply (WOS) space on site is 7 WOS. This is a total across all items. Slow moving items may be 13+ WOS, fast moving may be 2 WOS, but overall should be capped at 7 WOS.
* Additionally, the supplier can hold up to 7 WOS of empty bags at no charge, then release in minimal order quantity (MOQ) amounts as needed. It is important to note that the supplier views this as no more than 7 weeks storage for each item, not in aggregate. When any item has been sitting for 7 weeks, the supplier will auto ship to PFF and the item will be received in one week. This capability should help minimize on site FG inventory, and also help maximize order increments to take advantage of lower prices with higher order amounts. § For example, we can order 100,000 bags, bring in 20,000 and keep 80,000 at supplier. Then release in increments of 20,000 until remaining 80,000 are consumed.

## Notes and Observations


## Assumptions:


# <span style="color:orange"><center><b>Module imports and data loading</b></center></span>

In [2]:
from _GUROBI_TOOLS_.GUROBI_MODEL_BUILDING_TOOLS import *
from _NOTE_BOOK_UTILS import *



<IPython.core.display.Javascript object>

# <a id=Data-Display><span style="color:Green"><center> Data Display</center></span></a>

In [50]:
# display data for problem

kibble_data_tab = "Item Mapping and Pricing"
demand_in_pounds_tab = "Bag demand"
mainfile = 'IE 402 - Group project - PPF - Template - 2 items 12 weeks-1 (1).xlsx'

MOQS = pd.read_excel(mainfile, sheet_name=kibble_data_tab, 
                         skiprows=2, nrows=2,
                         usecols=[4])
kibble_price_breaks = pd.read_excel(mainfile, sheet_name=kibble_data_tab, 
                         skiprows=2, nrows=None,
                         usecols=list(range(5, 26)))

kibble_df = pd.read_excel(mainfile, sheet_name=kibble_data_tab, 
                         skiprows=None, nrows=None,
                         usecols=None)


WOS7 = pd.read_excel(mainfile, sheet_name=demand_in_pounds_tab, 
                          skiprows=5, nrows=None,
                          usecols=list(range(1,13))).values.tolist()[0]

kibble_demands_df = pd.read_excel(mainfile, sheet_name=demand_in_pounds_tab, 
                                  skiprows=1, nrows=2, header=1,
                                  usecols=list(range(1, 13)))

kibble_totals_df = pd.read_excel(mainfile, sheet_name=demand_in_pounds_tab, 
                                  skiprows=3, nrows=1, header=1,
                                  usecols=list(range(1, 13))).values.tolist()[0]


demand_df = pd.read_excel(mainfile, sheet_name=demand_in_pounds_tab, 
                          skiprows=None, nrows=None, 
                          usecols=None)

print("minimum order quantity")
MOQs = MOQS.values.flatten().tolist()
display(MOQ)
print("\nPrice breaks")
display(kibble_price_breaks)
print("\nfull kibble info")
display(kibble_df)
print("\n-----------------------------------")
print("\n7WOS")
display(WOS7)
print("\nWeekly Kibble demand")
display(kibble_demands_df.values)
print("\nWeekly Kibble totals")
display(kibble_totals_df)
print("\nDemand data")
display(demand_df)

minimum order quantity


,MOQ
0,15000
1,15000



Price breaks


,10000,15000,20000,25000,30000,40000,50000,70000,80000,100000,...,200000,250000,300000,350000,400000,450000,500000,750000,800000,1000000
0,NaN,1,NaN,NaN,0.846793,0.764952,0.715186,NaN,NaN,0.614208,...,0.573538,NaN,0.564907,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1,NaN,NaN,0.846793,0.764952,0.715186,NaN,NaN,0.614208,...,0.573538,NaN,0.564907,NaN,NaN,NaN,NaN,NaN,NaN,NaN



full kibble info


...

array([[    0.        ,     0.        ,     0.        ,     0.        ,
        15185.24238747,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ],
       [    0.        , 14488.88323258,     0.        ,     0.        ,
            0.        , 21733.32484887,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        , 21733.32484887]])


Weekly Kibble totals


[0.0,
 14488.88323258034,
 0.0,
 0.0,
 15185.242387473272,
 21733.324848870507,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 21733.324848870507]


Demand data


,WEEKLY PRODUCTION PLAN - #'s,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ITEM,2019-12-29 00:00:00,2020-01-05 00:00:00,2020-01-12 00:00:00,2020-01-19 00:00:00,2020-01-26 00:00:00,2020-02-02 00:00:00,2020-02-09 00:00:00,2020-02-16 00:00:00,2020-02-23 00:00:00,2020-03-01 00:00:00,2020-03-08 00:00:00,2020-03-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,10009,0,0,0,0,15185.242387,0,0,0,0,0,0,0,1265.436866,1265.436866,1265.436866,1265.436866,1265.436866,1265.436866
3,10010,0,14488.883233,0,0,0,21733.324849,0,0,0,0,0,21733.324849,4829.627744,4829.627744,4829.627744,4829.627744,4829.627744,4829.627744
4,TOTAL,0,14488.883233,0,0,15185.242387,21733.324849,0,0,0,0,0,21733.324849,6095.064610,6095.064610,6095.064610,6095.064610,6095.064610,6095.064610
5,7 WOS,51407.450469,51407.450469,36918.567236,36918.567236,36918.567236,43466.649698,27828.389459,33923.454069,40018.518678,46113.583288,52208.647898,58303.712508,NaN,NaN,NaN,NaN,NaN,NaN


[15000, 15000]

# <a id=Model-Formulation><center> <span style="color:blue"> Model Formulation</span> </center></a>
* [Paremeters and Sets](#Parameters-and-Sets)
* [Variables](#Variables)
* [Equations and Constraints](#Equations-and-Constraints)
* [Objective](#Objective)

## <a id=Parameters-and-Sets><span style="color:DarkBlue">Parameters and Sets:</span></a>

# Sets
### $\textbf{K}  \quad \quad \quad \text{set of kibble items for purchase/sale, } k \in \textbf{K}$ 
### $\textbf{E}  \quad \quad \quad \text{set of items for extruders \{1,2,3\}, } e \in \textbf{E}$ 
### $\textbf{P}  \quad \quad \quad \text{set of items for extruders \{1,2,3,4\}, } p \in \textbf{P}$ 
### $\textbf{W}  \quad \quad \quad \text{set of weeks the production is forcasted, \{1,...,12\}} w \in \textbf{W}$ 
### $\textbf{E}  \quad \quad \quad \text{set of items for extruders, \{1,2,3\}} e \in \textbf{E}$ 

# Parameters
### $\textbf{$Q_k$}  \quad \quad \quad \text{minimum order quantity for kibble $k$, } q \in \textbf{Q}$ 
### $\textbf{$W_k$}  \quad \quad \quad \text{unit weight for kibble $k$, } q \in \textbf{Q}$ 
### $R \quad \quad \quad \text{set of ranges for different discount prices = \{[5,10), [10, 15), [15, M]\}}, r \in R$
### $F_{k,r} \quad \quad \quad \text{Fixed charges for order amount $X_{w,k}$ such that r[0] $\leq X_{w,k}$ < r[1]}$
### $\textbf{$M$}  \quad \quad \quad \text{total demand expected over W weeks for all kibble }$
### $\textbf{$I_k$}  \quad \quad \quad \text{initial amount of kibble k on hand }$
### $\textbf{$U_{k}$}  \quad \quad \quad \text{per bag weight for kibble k}$

# Variables
### $\textbf{$X_{w,k}$}  \quad \quad \quad \text{amount of kibble k to order in week w, \{1,2,3\}}$ 
### $\textbf{$H_{w,k}$}  \quad \quad \quad \text{amount of kibble k on hand in week w}$ 
### $\textbf{$S_{w,k}$}  \quad \quad \quad \text{amount stored by supplier of kibble k in week w}$ 
### $\textbf{$O_{w,k}$}  \quad \quad \quad \text{binary variable for ordering kibble k in week w}$ 
### $\textbf{$P_{w,k,r}$}  \quad \quad \quad \text{binary binary 1 if r[0] $\leq X_{w,k}$ < r[1], 0 otherwise }$
### $\textbf{$D_{w,k}$}  \quad \quad \quad \text{demand for kibble k in week w }$
### $\textbf{$C_{w}$}  \quad \quad \quad \text{Total costs of ordering for week w }$
### $\textbf{$T$}  \quad \quad \quad \text{total costs after W weeks }$
### $\textbf{$?$}  \quad \quad \quad \text{? }$
### $\textbf{$?$}  \quad \quad \quad \text{? }$
### $\textbf{$?$}  \quad \quad \quad \text{? }$

# Objective: 
## $$\text{minimize (T} = (\sum_{w=1}^{\text{|W|}}\sum_{k=1}^{|K|}\sum_{r=1}^{|R|}(F_{k,r} \cdot P_{w,k,r})\cdot  O_{w,k})\text{)}$$
### $$\min(T)$$



# Constraints and Equations
#### Total cost calculation:
$$T = \sum_{w=1}^{|W|}\sum_{k=1}^{|K|}\sum_{r=1}^{|R|} (X_{w,k} \cdot F_{r})$$
$$? = \sum_{?=?}^{|?|}\sum_{?=1}^{|?|}\sum_{?=1}^{|?|} (?_{?,?} \cdot ?_{?})$$
#### MOQ constraint: must at least order the set minimum order amount each week
## $$X_{w,k} \geq \mu \cdot O_{w,k}, \forall \text{ $w,k$}$$
## $$X_{w,k} \leq M \cdot O_{w,k} + \mu, \forall \text{ $w,k$}$$


#### Amount on hand in the current week is dependent on what was ordered (x) and what was demanded (D) in the previous
$$X_{\text{w-1}} - D_{\text{w-1}} + H_{\text{w-1}} = H_{\text{w}}$$


#### Minimum amount on hand constraint: must have at least the minimum amount in the current week 
$$H_{\text{w}} = X_{\text{w-1}} \cdot O_{w-1} - D_{\text{w-1}} + K, \text{ w = 0}$$
$$H_{\text{w}} = X_{\text{w-1}} \cdot O_{w-1}  - D_{\text{w-1}} + H_{\text{w-1}}, \forall \text{ w > 0}$$
$$H_{\text{w}} \geq S_{\text{w }}, \forall \text{ w}$$      

#### only one price at a time constraint
$$ \sum_{r=1}^{|R|} (P_{w,k,r} ) == 1\quad \forall w,k$$

#### conditional pricing constraints
$$P_{\text{w},k,1} == 1 \implies P_{\text{w},2} == 0\text{ and }P_{\text{w},3} == 0$$
$$P_{\text{w},k,1} == 1 \implies X_{w,r} \leq r[1]$$
$$P_{\text{w},k,2} == 0 \implies X_{w,r} \leq r[1]$$
$$P_{\text{w},k,3} == 0 \implies X_{w,r} \leq r[1]$$

$$P_{\text{w},k,2} == 1 \implies P_{\text{w},1} == 0\text{ and }P_{\text{w},3} == 0$$
$$P_{\text{w},k,2} == 1 \implies X_{w,2} \geq r[2]$$
$$P_{\text{w},k,1} == 0 \implies X_{w,2} \geq r[2]$$
$$P_{\text{w},k,3} == 0 \implies X_{w,2} \geq r[2]$$


$$P_{\text{w},k,3} == 1 \implies P_{\text{w},1} == 0\text{ and }P_{\text{w},2} == 0$$
$$P_{\text{w},k,3} == 1 \implies X_{w,k,3} \geq r[3]$$
$$P_{\text{w},k,1} == 0 \implies X_{w,k,3} \geq r[3]$$
$$P_{\text{w},k,2} == 0 \implies X_{w,k,3} \geq r[3]$$

# latex templates
### $\textbf{$?$}  \quad \quad \quad \text{? }$
$$? = \sum_{?=?}^{|?|}\sum_{?=1}^{|?|}\sum_{?=1}^{|?|} (?_{?,?} \cdot ?_{?})$$

## <a id=Variables><span style="color:DarkBlue">Variables:</span></a>

### $X_{w,c} \quad  \text{     amount from warehouse w supplied to customer } c$ 

## <a id=Equations-and-Constraints><span style="color:DarkBlue">Equations and Constraints:</span></a>

>### <center><span style="font-size:30px;color:red"><b>Equation </b></span></center>

# $$0 \leq \quad \sum_{c=1}^{|C|}X_{w,c} \quad  \leq H_{w}, \forall w$$

## <a id=Objective><span style="color:green">Objective: </span></a>

# $$\min(T = N + M)$$

# <a id=Method-definitions><center>Method Definitions</center></a>

In [79]:
# process price breaks into a set of ranges/values
def generatePriceBreaks2(price_df, verbose=False):
    colvalues = list(price_df.columns)
    price_breaks = dict()
    print(colvalues)
    crange = list()
    for pbl in colvalues:
        if verbose:
            print(pbl)
            print(price_df[pbl].count())
        # every time we find a zero length row and the crange list is empty add it as the bottom
        if len(crange) == 0 and price_df[pbl].count() == 0:
            crange = [pbl]
        elif len(crange) == 0 and price_df[pbl].count() > 0:
            price_breaks[price_df.loc[0, pbl]] = [pbl]
        else:
            if price_df[pbl].count() > 0:
                crange.append(pbl)
                price_breaks[price_df.loc[0, pbl]] = crange
                crange = list()
    print(price_breaks)
    return price_breaks

# process price breaks into a set of ranges/values
def generatePriceBreaks(price_df, verbose=False):
    colvalues = list(price_df.columns)
    price_breaks = dict()
    print(colvalues)
    crange = list([0])
    r = 0             # keep track of how long it has been(iterations) since we last made a range
    for pbl in colvalues:
        if verbose:
            print(pbl)
            print(price_df[pbl].count())
        # when we have just added a range or on first run
        # just add the value as the lower bound
        # and increment the new range time
        if r == 0: 
                crange.append(pbl)
                r +=1
        # after the lower bound has been added right after the last range check for a empty 
        # column and if found add that as the upper bound
        # if not found continue looking and increment r
        else:
            if price_df[pbl].count() == 0:
                r += 1
            elif price_df[pbl].count() > 0:
                # set as upper bound and reset are since we just 
                # added a range
                #crange.append(pbl)
                price_breaks[price_df.loc[0, pbl]] = crange
                r = 1
                crange= list([pbl])
    print(price_breaks)
    return price_breaks

def convertlbsToBags(lbs, unit_weight):
    
    


# generate objective
def generate_obj(df, unit_cost_col, fr, Xws, Ows, Dws, Hws, Pw):
    expression = None
    print("--",len(df))
    for i in range(len(df)):
        if i == 0:
            order_cost = (fr[i,0] *Pw[i,0] + fr[i,1] *Pw[i,1] + fr[i,2] *Pw[i,2])
            expression =  (Hws[i])*df.loc[i, unit_cost_col] + order_cost
        else:
            order_cost = (fr[i,0] *Pw[i,0] + fr[i,1] *Pw[i,1] + fr[i,2] *Pw[i,2])
            expression +=  (Hws[i])*df.loc[i, unit_cost_col] + order_cost

    return expression
# generate constraints

price_break_dict = generatePriceBreaks(kibble_price_breaks, verbose=False)

[10000, 15000, 20000, 25000, 30000, 40000, 50000, 70000, 80000, 100000, 150000, 200000, 250000, 300000, 350000, 400000, 450000, 500000, 750000, 800000, 1000000]
{1: [0, 10000], 0.8467925308344243: [15000], 0.7649521118996762: [30000], 0.715186384620685: [40000], 0.614207951491766: [50000], 0.5735375180872322: [100000], 0.5649073244677186: [200000]}


In [75]:
for p in price_break_dict:
    if len(price_break_dict[p]) == 2:
        l = price_break_dict[p][0]
        u = price_break_dict[p][1]
        print("{p}:  [{l}, {u}]".format(p=p, l=l, u=u))
    else:
        size = price_break_dict[p][0]
        print("price: {p}, size: {l} -- {u}".format(p=p, l=l, u=u))
        

1:  [10000, 15000]
0.8467925308344243:  [15000, 30000]
0.7649521118996762:  [30000, 40000]
0.715186384620685:  [40000, 50000]
0.614207951491766:  [50000, 100000]
0.5735375180872322:  [100000, 200000]
0.5649073244677186:  [200000, 300000]


# <a id=implementation><center>Gurobi Implementation and Solution</center></a>

In [81]:
try:
    # instantiate model object 
    m = gp.Model("G_MOD")
 

    
    
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################
    W = 12              # number of weeks
    K = 2               # number of different types of kibble
    R = len(price_break_dict)
    
    fixed_costs = list(price_break_dict.keys())
    fixed_ranges = list(price_break_dict.values())
    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################
    #                               SUPPLY/ORDERING
    # ordering amounts
    Xwk = m.addVars(W, K, vtype=GRB.CONTINUOUS, name="X_week_kibble", lb=MOQs[0])
    
    # Binary decision variable for week w and kibble k
    Owk = m.addVars(W, K, vtype=GRB.BINARY, name="Order_week_kibble", lb=0, ub=1)  
    
    # Binary amount pricing variable
    Pwkr = m.addVars(W, K, R, vtype=GRB.BINARY, name="P", lb=0, ub=1) 
    
   
    # fixed costs for each range
    Fwkr = m.addVars(W, K, R, vtype=GRB.CONTINUOUS, name="FixedRate_week_kibble", lb=.001)
    
    # amount on hand in a week amounts
    Hwk = m.addVars(W, K, vtype=GRB.CONTINUOUS, name="OnHand_week_kibble", lb=MOQs[0])   
    
    #                               DEMAND
    # demand amounts
    Dwk = m.addVars(W, K, vtype=GRB.CONTINUOUS, name="Demand_week_kibble", lb=0)
    
    #                               PRICEING-
    
    
    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################    
    
    #########################################################################################
    ################################## Constraint set up ####################################
    #########################################################################################
    
    
    #########################################################################################
    ################################## SOLVE:OPTIMIZE #######################################
    #########################################################################################    
    
    
    m.optimize()
    
    #########################################################################################
    ################################## Display Results ######################################
    #########################################################################################    
    displayDecisionVars(m, end_sentinel="11,1")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj: {:.2f}'.format(m.ObjVal))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 432 columns and 0 nonzeros
Model fingerprint: 0x2fd8bc58
Variable types: 240 continuous, 192 integer (192 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e-03, 2e+04]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
X_week_kibble[0,0] 15000.00000
X_week_kibble[0,1] 15000.00000
X_week_kibble[1,0] 15000.00000
X_week_kibble[1,1] 15000.00000
X_week_kibble[2,0] 15000.00000
X_week_kibble[2,1] 15000.00000
X_week_kibble[3,0] 15000.00000
X_week_kibble[3,1] 15000.00000
X

# <a id=solution><span style="color:crimson"><center>Solution Discussion</center></a>

> The solution....

In [ ]:
# save the notebook as a pdf
# to_PDF("notebook_title")